In [1]:
pip install tensorflow torch pyro-ppl

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import torch
import pyro
import pyro.distributions as dist
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
import pyro.distributions as dist
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder,OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.compose import ColumnTransformer

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
train_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/train.csv')

In [5]:
train_data.head()

,id,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes
0,0,Mystery Matters,Episode 98,NaN,True Crime,74.81,Thursday,Night,NaN,0.0,Positive,31.41998
1,1,Joke Junction,Episode 26,119.80,Comedy,66.95,Saturday,Afternoon,75.95,2.0,Negative,88.01241
2,2,Study Sessions,Episode 16,73.90,Education,69.97,Tuesday,Evening,8.97,0.0,Negative,44.92531
3,3,Digital Digest,Episode 45,67.17,Technology,57.22,Monday,Morning,78.70,2.0,Positive,46.27824
4,4,Mind & Body,Episode 86,110.51,Health,80.07,Monday,Afternoon,58.68,3.0,Neutral,75.61031


In [6]:
# Replace null values with Median
train_data['Episode_Length_minutes'].fillna(train_data['Episode_Length_minutes'].median(), inplace=True)
train_data['Guest_Popularity_percentage'].fillna(train_data['Guest_Popularity_percentage'].median(), inplace=True)

<ipython-input-6-d503df9b77f4>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_data['Episode_Length_minutes'].fillna(train_data['Episode_Length_minutes'].median(), inplace=True)
<ipython-input-6-d503df9b77f4>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when

In [7]:
# remove id column
train_data = train_data.drop('id', axis=1)
train_data = train_data.drop('Podcast_Name', axis=1)

In [8]:
# Remove the word 'Episode' from the Episode_Title column
train_data['Episode_Title'] = train_data['Episode_Title'].str.replace('Episode', '', regex=False)
# Change the Episode_Title name to Episode number
train_data.rename(columns={'Episode_Title': 'Episode_Number'}, inplace=True)
train_data['Episode_Number'] = train_data['Episode_Number'].astype('float64')

In [9]:
# Split features and target
X = train_data.drop(columns=['Listening_Time_minutes'])
y = train_data['Listening_Time_minutes']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Identify column types
numerical_cols = ['Episode_Number', 'Episode_Length_minutes',
                  'Host_Popularity_percentage', 'Guest_Popularity_percentage',
                  'Number_of_Ads']
numerical_cols = [col for col in numerical_cols if col in X.columns]
categorical_cols = [col for col in X.columns if col not in numerical_cols]

# Define transformers with scaling and one-hot encoding
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_cols)
    ]
)

# Fit + transform the training and test sets
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

# Convert all to float32
X_train_np = X_train_processed.astype(np.float32)
X_test_np = X_test_processed.astype(np.float32)
y_train_np = y_train.to_numpy(dtype=np.float32)
y_test_np = y_test.to_numpy(dtype=np.float32)

print(f"Training set size: {X_train_np.shape[0]} samples")
print(f"Testing set size: {X_test_np.shape[0]} samples")


Training set size: 600000 samples
Testing set size: 150000 samples


In [10]:
X_train_np.shape

(600000, 29)

In [11]:
# Option 2: Impute (fill) NaNs, e.g. with mean
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean')
X_train_np = imputer.fit_transform(X_train_np)


In [12]:
# Initialize parameters
np.random.seed(42)
n_features = X_train_np.shape[1]
weights = np.random.randn(n_features, 1).astype(np.float32) * 0.01
bias = np.float32(0.0)
noise_variance = np.float32(100.0)  # Match target variance
prior_variance = np.float32(100.0)  # Stronger regularization
learning_rate = np.float32(2e-5)
n_iterations = 1000
batch_size = 1024
tol = 1e-4
clip_norm = 10.0  # Gradient clipping threshold
max_lr_reductions = 5  # Limit learning rate reductions

In [13]:
# Clear Pyro parameter store
pyro.clear_param_store()
print("WARNING: If shape errors persist, restart the Jupyter kernel to clear cached parameters.")

# Normalize target
y_scaler = StandardScaler()
y_train_np = y_scaler.fit_transform(y_train_np.reshape(-1, 1)).flatten()
y_test_np = y_scaler.transform(y_test_np.reshape(-1, 1)).flatten()

# Validation split
X_train_sub, X_val, y_train_sub, y_val = train_test_split(X_train_np, y_train_np, test_size=0.2, random_state=42)

# Convert to torch tensors
X_train_sub = torch.from_numpy(X_train_sub).float()
y_train_sub = torch.from_numpy(y_train_sub).float()
X_val = torch.from_numpy(X_val).float()
y_val = torch.from_numpy(y_val).float()
X_test = torch.from_numpy(X_test_np).float()
y_test = torch.from_numpy(y_test_np).float()

# Subsample validation set
val_sample_indices = torch.randperm(X_val.shape[0])[:10000]
X_val_sub = X_val[val_sample_indices]
y_val_sub = y_val[val_sample_indices]

# Hyperparameters
n_iterations = 500
batch_size = 5000
learning_rate = 7e-4
hidden_units = 48
noise_variance = 2.0
tol = 5e-3
lr_schedule_step = 300
lr_schedule_factor = 0.9

# Define BNN model
def model(X, y=None):
    n_features = X.shape[1]
    w1 = pyro.sample("w1", dist.Normal(0., 1.).expand([n_features, hidden_units]).to_event(2))
    b1 = pyro.sample("b1", dist.Normal(0., 1.).expand([hidden_units]).to_event(1))
    w2 = pyro.sample("w2", dist.Normal(0., 1.).expand([hidden_units, 1]).to_event(2))
    b2 = pyro.sample("b2", dist.Normal(0., 1.).expand([1]).to_event(1))

    assert w1.shape == (n_features, hidden_units), f"w1 shape mismatch: {w1.shape}"
    assert b1.shape == (hidden_units,), f"b1 shape mismatch: {b1.shape}"
    assert w2.shape == (hidden_units, 1), f"w2 shape mismatch: {w2.shape}"
    assert b2.shape == (1,), f"b2 shape mismatch: {b2.shape}"

    h = torch.relu(X @ w1 + b1)
    out = h @ w2 + b2

    with pyro.plate("data", X.shape[0]):
        pyro.sample("obs", dist.Normal(out.squeeze(), noise_variance), obs=y)

# Define guide
def guide(X, y=None):
    n_features = X.shape[1]
    w1_loc = pyro.param("w1_loc_v2", torch.zeros(n_features, hidden_units))
    w1_scale = pyro.param("w1_scale_v2", torch.ones(n_features, hidden_units), constraint=dist.constraints.positive)
    b1_loc = pyro.param("b1_loc_v2", torch.zeros(hidden_units))
    b1_scale = pyro.param("b1_scale_v2", torch.ones(hidden_units), constraint=dist.constraints.positive)
    w2_loc = pyro.param("w2_loc_v2", torch.zeros(hidden_units, 1))
    w2_scale = pyro.param("w2_scale_v2", torch.ones(hidden_units, 1), constraint=dist.constraints.positive)
    b2_loc = pyro.param("b2_loc_v2", torch.zeros(1))
    b2_scale = pyro.param("b2_scale_v2", torch.ones(1), constraint=dist.constraints.positive)

    assert w1_loc.shape == (n_features, hidden_units), f"w1_loc shape mismatch: {w1_loc.shape}"
    assert b1_loc.shape == (hidden_units,), f"b1_loc shape mismatch: {b1_loc.shape}"
    assert w2_loc.shape == (hidden_units, 1), f"w2_loc shape mismatch: {w2_loc.shape}"
    assert b2_loc.shape == (1,), f"b2_loc shape mismatch: {b2_loc.shape}"

    pyro.sample("w1", dist.Normal(w1_loc, w1_scale).to_event(2))
    pyro.sample("b1", dist.Normal(b1_loc, b1_scale).to_event(1))
    pyro.sample("w2", dist.Normal(w2_loc, w2_scale).to_event(2))
    pyro.sample("b2", dist.Normal(b2_loc, b2_scale).to_event(1))

# Training setup
optimizer = Adam({"lr": learning_rate})
svi = SVI(model, guide, optimizer, loss=Trace_ELBO())
loss_history = []
val_loss_history = []

# Training loop
print("Starting training loop...")
for i in range(n_iterations):
    if i % lr_schedule_step == 0 and i > 0:
        learning_rate *= lr_schedule_factor
        optimizer = Adam({"lr": learning_rate})
        svi = SVI(model, guide, optimizer, loss=Trace_ELBO())
        print(f"[Iter {i:04d}] Reduced learning rate to {learning_rate:.2e}")

    indices = torch.randperm(X_train_sub.shape[0])
    X_train_sub_shuffled = X_train_sub[indices]
    y_train_sub_shuffled = y_train_sub[indices]

    mini_batch_count = 0
    elbo = 0
    for batch_start in range(0, X_train_sub.shape[0], batch_size):
        batch_end = min(batch_start + batch_size, X_train_sub.shape[0])
        X_batch = X_train_sub_shuffled[batch_start:batch_end]
        y_batch = y_train_sub_shuffled[batch_start:batch_end]

        elbo += svi.step(X_batch, y_batch) / (X_train_sub.shape[0] / batch_size)
        mini_batch_count += 1

    val_elbo = 0
    for batch_start in range(0, X_val_sub.shape[0], batch_size):
        batch_end = min(batch_start + batch_size, X_val_sub.shape[0])
        X_batch = X_val_sub[batch_start:batch_end]
        y_batch = y_val_sub[batch_start:batch_end]
        val_elbo += svi.evaluate_loss(X_batch, y_batch) / (X_val_sub.shape[0] / batch_size)

    loss_history.append(elbo)
    val_loss_history.append(val_elbo)

    if i % 50 == 0 or i < 5:
        print(f"[Iter {i:04d}] Train ELBO: {elbo:.4f}, Val ELBO: {val_elbo:.4f}, LR: {learning_rate:.2e}")

    if i > 50 and abs(loss_history[-10] - loss_history[-1]) < tol:
        print(f"[INFO] Converged at iteration {i}")
        break

    print(f"Iteration {i + 1}/{n_iterations} completed with {mini_batch_count} mini-batches.")

# Predictive distribution
def predict(X, n_samples=100):
    predictive = pyro.infer.Predictive(model, guide=guide, num_samples=n_samples)
    samples = predictive(X)["obs"]
    return samples.mean(dim=0).squeeze().numpy()

# Evaluate on test set
y_pred = predict(X_test)
y_pred = y_scaler.inverse_transform(y_pred.reshape(-1, 1)).flatten()
y_test_np = y_scaler.inverse_transform(y_test.numpy().reshape(-1, 1)).flatten()
mse = mean_squared_error(y_test_np, y_pred)
r2 = r2_score(y_test_np, y_pred)
print(f"Test MSE: {mse:.4f}")
print(f"Test R²: {r2:.4f}")

# Residual plot
#plt.scatter(y_pred, y_test_np - y_pred, alpha=0.1)
#plt.axhline(0, color='r')
#plt.xlabel("Predicted Listening Time (minutes)")
#plt.ylabel("Residuals (Actual - Predicted)")
#plt.title("Residual Plot")
#plt.show()

# Prediction intervals
n_samples = 500
y_samples = predict(X_test, n_samples=n_samples)
y_samples = y_scaler.inverse_transform(y_samples.reshape(-1, 1)).reshape(n_samples, -1)
intervals = 1.96 * y_samples.std(axis=0)
print(f"Sample 95% CI (first 5): {intervals[:5]}")

Starting training loop...
[Iter 0000] Train ELBO: 155114.9300, Val ELBO: 299843.0684, LR: 7.00e-04
Iteration 1/500 completed with 96 mini-batches.
[Iter 0001] Train ELBO: 140556.5712, Val ELBO: 96393.1840, LR: 7.00e-04
Iteration 2/500 completed with 96 mini-batches.
[Iter 0002] Train ELBO: 124812.4526, Val ELBO: 236792.9739, LR: 7.00e-04
Iteration 3/500 completed with 96 mini-batches.
[Iter 0003] Train ELBO: 137242.3239, Val ELBO: 115510.5187, LR: 7.00e-04
Iteration 4/500 completed with 96 mini-batches.
[Iter 0004] Train ELBO: 122955.9965, Val ELBO: 108080.1381, LR: 7.00e-04
Iteration 5/500 completed with 96 mini-batches.
Iteration 6/500 completed with 96 mini-batches.
Iteration 7/500 completed with 96 mini-batches.
Iteration 8/500 completed with 96 mini-batches.
Iteration 9/500 completed with 96 mini-batches.
Iteration 10/500 completed with 96 mini-batches.
Iteration 11/500 completed with 96 mini-batches.
Iteration 12/500 completed with 96 mini-batches.
Iteration 13/500 completed with

In [14]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/test.csv')
df = df.drop(["id", "Podcast_Name"], axis=1)

df["Episode_Title"] = df["Episode_Title"].str.replace("Episode ", "", regex=False)
df["Episode_Number"] = df["Episode_Title"].str.extract(r"(\d+)$").astype(float)
df = df.drop("Episode_Title", axis=1)

numerical_cols = [
    "Episode_Length_minutes",
    "Host_Popularity_percentage",
    "Guest_Popularity_percentage",
    "Number_of_Ads"
]

for col in numerical_cols:
    median_val = df[col].median()
    df[col] = df[col].fillna(median_val)

categorical_cols = ["Genre", "Publication_Day", "Publication_Time", "Episode_Sentiment"]
df = pd.get_dummies(df, columns=categorical_cols)

numerical_cols = ["Episode_Number"] + numerical_cols
scaler = StandardScaler()
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

print(df.head())


   Episode_Length_minutes  Host_Popularity_percentage  \
0               -0.001908                   -0.944340   
1               -0.002233                    0.505836   
2               -0.001971                    0.357234   
3               -0.001676                   -1.587260   
4               -0.001950                   -0.070651   

   Guest_Popularity_percentage  Number_of_Ads  Episode_Number  Genre_Business  \
0                     0.035627      -0.083252        0.769228           False   
1                     0.036803      -0.317204       -1.013125           False   
2                     1.766796      -0.317204       -1.440889           False   
3                    -0.026284       0.150699        0.769228           False   
4                    -1.611295       0.150699       -0.050654           False   

   Genre_Comedy  Genre_Education  Genre_Health  Genre_Lifestyle  ...  \
0         False             True         False            False  ...   
1         False           

In [15]:
# Simplified prediction function
def predict_listening_time(test_df, model, guide, y_scaler, n_samples=100):
    """
    Predict Listening_Time_minutes for a test DataFrame using the trained BNN model.

    Args:
        test_df (pd.DataFrame): Test data with same features as training.
        model (callable): Trained BNN model function.
        guide (callable): Trained BNN guide function.
        y_scaler (StandardScaler): Scaler for target (Listening_Time_minutes).
        n_samples (int): Number of Monte Carlo samples for prediction (default: 100).

    Returns:
        pd.DataFrame: DataFrame containing 'ID' and 'Predicted Listening Time'.
    """
    # Convert DataFrame to NumPy
    X_test_np = test_df.to_numpy().astype(np.float32)

    # Convert to torch tensor
    X_test = torch.from_numpy(X_test_np).float()

    # Predictive distribution
    def predict(X, n_samples=n_samples):
        predictive = pyro.infer.Predictive(model, guide=guide, num_samples=n_samples)
        samples = predictive(X)["obs"]
        return samples.mean(dim=0).squeeze().numpy()

    # Get predictions (normalized)
    y_pred = predict(X_test, n_samples=n_samples)

    # Denormalize predictions
    y_pred = y_scaler.inverse_transform(y_pred.reshape(-1, 1)).flatten()

    # Create IDs starting from 750000
    ids = np.arange(750000, 750000 + len(y_pred))

    # Create DataFrame to store IDs and predictions
    results_df = pd.DataFrame({
        'id': ids,
        'Listening_Time_minutes': y_pred
    })

    # Save the results to a CSV file
    results_df.to_csv("predictions6.csv", index=False)

    return results_df  # Optionally return the DataFrame

# Example usage:
# Assume model, guide, y_scaler are defined and test data is available in df
predictions_df = predict_listening_time(df, model, guide, y_scaler, n_samples=100)

print(predictions_df.head())  # Print first few rows of the predictions DataFrame


       id  Listening_Time_minutes
0  750000               31.941841
1  750001               71.183640
2  750002               58.668865
3  750003                5.741311
4  750004               52.789505
